# Traitement des données de raw_flows pour regroupement par destination et analyse des produits et tonnages expédiés

In [1]:
import pandas as pd
import pandas.core.frame as pf


In [2]:
# Aller chercher les données de ports via l'API (à rajouter dans le load_data.sh)

#curl -o data/navigo_ports.csv "data.portic.fr/api/ports/?format=csv&srid=4326"


ports = pd.read_csv("http://data.portic.fr/api/ports/?format=csv&srid=4326", encoding='utf8')


In [3]:
# Récupérer les raw_flows
#http://data.portic.fr/api/rawflows/?format=csv&date=1789&params=departure_uhgs_id,departure_fr,destination_uhgs_id,destination_fr,destination_partner_balance_supp_1789,destination_substate_1789_fr,destination_state_1789_fr,tonnage,outdate_fixed,commodity_purpose,commodity_purpose2,commodity_purpose3,commodity_purpose4,all_cargos,nb_cargo

#curl -o data/raw_flows_1789.csv "data.portic.fr/api/rawflows/?format=csv&date=1789&params=departure_uhgs_id,departure_fr,destination_uhgs_id,destination_fr,destination_partner_balance_supp_1789,destination_substate_1789_fr,destination_state_1789_fr,tonnage,outdate_fixed,commodity_purpose,commodity_purpose2,commodity_purpose3,commodity_purpose4,all_cargos,nb_cargo"
raw_flows = pd.read_csv("./raw_flows_1789.csv", encoding='utf8', sep=',')


type(raw_flows) #list

#df_dix_ans_smog = pd.DataFrame(dix_ans_smog, columns=['destination', '1781', '1782', '1785', '1786', '1787', '1788', '1789', '1790', '1791', 'Facade'])
print(raw_flows.shape)

print(raw_flows.columns)



(10081, 16)
Index(['Unnamed: 0', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',
       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',
       'commodity_purpose4', 'all_cargos', 'nb_cargo'],
      dtype='object')


    'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',
       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',
       'commodity_purpose4', 'all_cargos', 'nbproduits'

In [4]:
# Filtrer

dunkerque = raw_flows.query('departure_fr=="Dunkerque"')
print(dunkerque.shape)
dunkerque = dunkerque.query('destination_partner_balance_supp_1789=="Etranger"') #Pas la France
dunkerque = dunkerque.query('destination_state_1789_fr!="Grande-Bretagne"') #Pas le smogglage vers l'angleterre
dunkerque = dunkerque.query('destination_uhgs_id!="C0000009"') #Afrique
dunkerque = dunkerque.query('destination_uhgs_id!="A0146289"') #Islande
dunkerque = dunkerque.query('destination_uhgs_id!="B0000715"')  #Terre-Neuve
dunkerque = dunkerque.query('commodity_purpose!="Lège"')  #pas à vide (lest)
dunkerque = dunkerque.query('commodity_purpose!="pêche"')  #pas pour la pêche
print(dunkerque.shape) #(193, 16)


print(dunkerque[['all_cargos']])

(2539, 16)
(193, 16)
                                             all_cargos
65    [{'quantity': None, 'quantity_u': None, 'commo...
131   [{'quantity': None, 'quantity_u': None, 'commo...
132   [{'quantity': None, 'quantity_u': None, 'commo...
133   [{'quantity': None, 'quantity_u': None, 'commo...
134   [{'quantity': None, 'quantity_u': None, 'commo...
...                                                 ...
9373  [{'quantity': None, 'quantity_u': None, 'commo...
9580  [{'quantity': None, 'quantity_u': None, 'commo...
9583  [{'quantity': None, 'quantity_u': None, 'commo...
9707  [{'quantity': None, 'quantity_u': None, 'commo...
9831  [{'quantity': None, 'quantity_u': None, 'commo...

[193 rows x 1 columns]


## Premier apercu des destinations d'export de Dunkerque vers l'étranger, hors GB, hors pêche, hors navires à vide. 

In [5]:


gdata = pf.DataFrame({'count' : dunkerque.groupby( [ "destination_state_1789_fr","destination_uhgs_id"] ).size()}).reset_index()
print(gdata)

pf.DataFrame(dunkerque.groupby([ "destination_state_1789_fr","destination_uhgs_id"])['tonnage'].agg(['sum','count'])).reset_index()


   destination_state_1789_fr destination_uhgs_id  count
0                   Autriche            A0041886     17
1                   Autriche            A0046492      1
2                   Autriche            A0049808     56
3                      Brême            A0691896      1
4                   Danemark            A0912818      1
5                   Danemark            A1964976      5
6                    Espagne            A0079352      7
7                    Espagne            B0000056      1
8      Etats-Unis d'Amérique            B2827324      1
9                   Hambourg            A0743522      3
10                  Portugal            A0360085      1
11                  Portugal            A0364551      1
12                  Portugal            A0374720      6
13           Provinces-Unies            A0618784      4
14           Provinces-Unies            A0620777     10
15           Provinces-Unies            A0622825      1
16           Provinces-Unies            A0624785

,destination_state_1789_fr,destination_uhgs_id,sum,count
0,Autriche,A0041886,516.0,17
1,Autriche,A0046492,60.0,1
2,Autriche,A0049808,3570.0,56
3,Brême,A0691896,70.0,1
4,Danemark,A0912818,35.0,1
5,Danemark,A1964976,134.0,5
6,Espagne,A0079352,943.0,7
7,Espagne,B0000056,120.0,1
8,Etats-Unis d'Amérique,B2827324,350.0,1
9,Hambourg,A0743522,435.0,3


In [7]:
import csv
import json

by_pointcalls = []


for _, row in dunkerque.iterrows():
  if row['all_cargos'] != '':
    truc = row['all_cargos']
    #print(truc)
    truc = truc.replace("'", '"').replace("None", 'null')
    #.replace("bœ\x9c\x9cuf", "boeuf")
    cargos = json.loads(truc)
    #print(type(cargos))
    for cargo in cargos:
      by_pointcalls.append({
        "cargo": cargo['commodity_purpose'],
        "cargo_standardized_fr": cargo['commodity_standardized_fr'],
        **row
      })
  else:
    by_pointcalls.append({
      "cargo": "",
      "cargo_standardized_fr": "",
      **row
    })



In [15]:
print(dunkerque.columns)
print(['cargo', 'cargo_standardized_fr']+dunkerque.columns.tolist())
exports_dunkerque = pd.DataFrame(by_pointcalls, columns=['cargo', 'cargo_standardized_fr']+dunkerque.columns.tolist())

exports_dunkerque = exports_dunkerque.drop(['Unnamed: 0', 'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3', 'commodity_purpose4', 'all_cargos'], axis=1)

Index(['Unnamed: 0', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',
       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',
       'commodity_purpose4', 'all_cargos', 'nb_cargo'],
      dtype='object')
['cargo', 'cargo_standardized_fr', 'Unnamed: 0', 'departure_uhgs_id', 'departure_fr', 'destination_uhgs_id', 'destination_fr', 'destination_partner_balance_supp_1789', 'destination_substate_1789_fr', 'destination_state_1789_fr', 'tonnage', 'outdate_fixed', 'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3', 'commodity_purpose4', 'all_cargos', 'nb_cargo']
